In [47]:
import os
import json
import csv
import sys
import datetime
import time
import numpy as np
import matplotlib.pyplot as plt
import librosa
import tensorflow as tf
import keras_tuner
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow import keras
from keras_tuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [2]:
csv_path = "csv_fma_10secs_data.csv"

sr=22050

csv.field_size_limit(sys.maxsize)

131072

In [58]:
currentDateAndTime = datetime.datetime.now()

LOG_DIR = currentDateAndTime.strftime("%H_%M_%S")

print(LOG_DIR)

00_28_36


In [3]:
#Read MFCC data from CSV
def csv_read_data(csv_path):
    # Load data from CSV file
    with open(csv_path, 'r') as csvfile:
        reader = csv.reader(csvfile)

        next(reader)

        # Initialize lists to hold genre and MFCC data
        genres = []
        mfcc = []

        # Iterate over each row of the CSV file
        for row in reader:
            # Extract genre and MFCC data from the row
            genre = int(row[0])
            mfcc_data = json.loads(row[1])

            # Append genre and MFCC data to lists
            genres.append(genre)
            mfcc.append(mfcc_data)

    # Convert lists to numpy arrays
    X = np.array(mfcc)
    y = np.array(genres)

    return X, y

In [4]:
#Use stratified split to seperate data into training, validation and test

def stratified_split_dataset(inputs, targets, split_size):
    sss = StratifiedShuffleSplit(n_splits=1, test_size=split_size, random_state=42)
    
    train_val_indices, test_indices = next(sss.split(inputs, targets))
    inputs_train_val, targets_train_val = inputs[train_val_indices], targets[train_val_indices]
    
    sss_train = StratifiedShuffleSplit(n_splits=1, test_size=split_size, random_state=43)
    train_indices, val_indices = next(sss_train.split(inputs_train_val, targets_train_val))
    
    inputs_train, targets_train = inputs_train_val[train_indices], targets_train_val[train_indices]
    inputs_val, targets_val = inputs_train_val[val_indices], targets_train_val[val_indices]
    inputs_test, targets_test = inputs[test_indices], targets[test_indices]
    
    #Needed for compatibility reasons
    inputs_train = inputs_train[...,np.newaxis]
    inputs_val = inputs_val[...,np.newaxis]
    inputs_test=  inputs_test[..., np.newaxis]
    
    
    return inputs_train, inputs_val, inputs_test, targets_train, targets_val, targets_test

In [5]:
inputs, targets = csv_read_data(csv_path)

In [6]:
#Splitting data
Xtrain, Xval, Xtest, ytrain, yval, ytest =stratified_split_dataset(inputs, targets, 0.2)

In [7]:
input_shape = (Xtrain.shape[1], Xtrain.shape[2], 1)

In [14]:
print(input_shape)

(417, 40, 1)


In [86]:
def call_existing_code(units, activation, lr, dropout, layers):
    model = keras.Sequential()
    
    model.add(tf.keras.layers.Conv2D(units, (3,3), activation=activation, input_shape=(417, 40, 1)),)
    model.add(tf.keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    
    
    for i in range (layers):
        model.add(tf.keras.layers.Conv2D(units, (3,3), activation=activation, input_shape=(417, 40, 1)),)
        model.add(tf.keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='same'))
        model.add(tf.keras.layers.BatchNormalization())
        if dropout:
            model.add(tf.keras.layers.Dropout(rate=0.3))
        
    
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units, activation='relu')) 
    model.add(tf.keras.layers.Dense(len(np.unique(targets)), activation='softmax'))

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [87]:
def build_model(hp):
    units = hp.Int("units", min_value=32, max_value=128, step=32)
    activation = hp.Choice("activation", ["relu", "tanh"])
    dropout = hp.Boolean("dropout")
    layers = hp.Int("n_layers", min_value=1, max_value=3, step=1)
    lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    # call existing model-building code with the hyperparameter values.
    model = call_existing_code(
        units=units, activation=activation, lr=lr, dropout=dropout, layers = layers
    )
    return model

In [100]:
build_model(keras_tuner.HyperParameters())

In [105]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=5000,
    executions_per_trial=2,
    overwrite=True,
    directory="./trials/tb",
)

In [106]:
tuner.search_space_summary()


Search space summary
Default search space size: 5
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
n_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [107]:
tuner.search(
    Xtrain,
    ytrain,
    validation_data=(Xval, yval),
    epochs=50,
    callbacks=[keras.callbacks.TensorBoard("./trials/tb_logs"),tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)] 
)


Trial 5 Complete [00h 01m 18s]
val_accuracy: 0.6228280365467072

Best val_accuracy So Far: 0.6384062469005585
Total elapsed time: 00h 05m 02s
INFO:tensorflow:Oracle triggered exit


In [108]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print out the values of the best hyperparameters
for hp in best_hps.values:
    print(f"{hp}: {best_hps.get(hp)}")

units: 96
activation: relu
dropout: False
n_layers: 2
lr: 0.0018308334177090633


In [92]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build(input_shape=(417, 40, 1))
best_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 415, 38, 32)       320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 208, 19, 32)      0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 208, 19, 32)      128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 206, 17, 32)       9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 103, 9, 32)       0         
 2D)                                                             
                                                        